In [1]:
import pandas as pd
import numpy as np


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk 
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import precision_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from config import TRACKING_URI, EXPERIMENT_NAME
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

In [ ]:
def __get_data():
    DATA_NAME = 'wikipedia_pre_clean'
    train = pd.read_csv("./data/train_" + DATA_NAME + ".csv")
    test = pd.read_csv("./data/test_" + DATA_NAME + ".csv")
    # cleaning data and preparing
    Y = train["toxic"]
    X = train["comment_text"]
    Y_test = test["toxic"]
    X_test= test["comment_text"]
    return X,Y, X_test, Y_test, DATA_NAME

In [ ]:
def objective(params):
    metrics = [roc_auc_score, precision_score]
    classifier = LogisticRegression()
    cv = cross_validate(classifier, X_train, y_train, scoring = metrics)
    scores = {metric: cv[f'test_{metric}'] for metric in metrics}
    # log all the stuff here
    mlflow.log_metric("ROC", scores[0])
    mlflow.log_metric("Precision", scores[1])

    mlflow.sklearn.log_model(classifier.fit(X, Y))
    return scores['some_loss'].mean()

space = hp.choice(...)
trials = SparkTrials(parallelism = ...)
with mlflow.start_run() as run:
    best_result = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 100, trials = trials)